<a href="https://colab.research.google.com/github/tramyynt/Vehicle-Routing-Problemn/blob/main/Vehicle_Routing_KU_Leuven.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np 
import pandas as pd
import numpy.linalg as LA
import os
import glob
import matplotlib.pyplot as plt
import seaborn as sns
import re
from datetime import datetime
import time
import json
import math
from sklearn.cluster import KMeans
import itertools

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
path = 'X-n819-k171'

In [ ]:
f = open('/content/drive/MyDrive/jsons/' + path + '.json')
data = json.load(f)
for k,v in data.items():
    print(str(k) + ' : ' + str(v))

name : X-n819-k171
dimension : 819
capacity : 358
depot : [500, 500]
id : ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89', '90', '91', '92', '93', '94', '95', '96', '97', '98', '99', '100', '101', '102', '103', '104', '105', '106', '107', '108', '109', '110', '111', '112', '113', '114', '115', '116', '117', '118', '119', '120', '121', '122', '123', '124', '125', '126', '127', '128', '129', '130', '131', '132', '133', '134', '135', '136', '137', '138', '139', '140', '141', '142', '143', '144', '145', '146', '147', '148

In [ ]:
def dump(obj):
  for attr in dir(obj):
    print("obj.%s = %r" % (attr, getattr(obj, attr)))

In [ ]:
def distance(point1, point2):
    return math.sqrt((point1.x - point2.x) ** 2 + (point1.y - point2.y) ** 2)

In [ ]:
def cluster_customers(num_clusters, customers):
    kmeans = KMeans(n_clusters=num_clusters,
                    init='k-means++',   # 'random', 'k-means++'
                    n_init=10,
                    max_iter=300,
                    tol=1e-04,
                    random_state=0)

    coords = np.array([[c.x, c.y] for c in customers])
    y_km = kmeans.fit_predict(coords)

    cluster_labels = np.unique(y_km)
    n_clusters = cluster_labels.shape[0]
    centroids = kmeans.cluster_centers_
    #print('clusters: %s' % cluster_labels)
    #print('centroid: %s' % centroids)

    return y_km, centroids

In [ ]:
def init_vehicles(depot, centroids, clusters, customers, max_capacity):
  
    #Initializes and sorts the cluster centroids(i.e. vehicles) by the nearest order of the distance between the depot and centroid.

    # Calculate the Euclidean distance between depot and each centroid.
    ordered_vehicles = []
    i = 0
    for centroid in centroids:
        # Get the customers in a cluster
        customers_in_cluster = []
        customers_array_in_cluster = np.array(customers)[clusters == i]
        for c in customers_array_in_cluster:
            customers_in_cluster.append(Customer(c.id, c.x, c.y, c.demand))

        dist = distance(depot, Point(centroid[0], centroid[1]))
        vehicle = Vehicle(i, max_capacity, 0, centroid[0], centroid[1], customers_in_cluster, dist)
        ordered_vehicles.append(vehicle)
        i += 1


    ordered_vehicles = sorted(ordered_vehicles, key=lambda x: x.initialDistance, reverse=True)

    return ordered_vehicles

In [ ]:
def assign_customers_to_vehicles(customer_list, vehicles, max_capacity):
    #Assigns the customers to vehicles.
    #One customer will be allocated only into one vehicle.
  
    customers = customer_list.copy()
    vehicles_ = []

    for vehicle in vehicles:
        ordered_customers_tuple = []
        assigned_customers = []
        customers_in_cluster = vehicle.customers
        remaining_capacity = vehicle.capacity

        # Assign customers in the cluster first
        for customer_in_cluster in customers_in_cluster:
            if remaining_capacity <= 0:
                break
            for customer in customers:
                new_remaining_capacity = remaining_capacity - customer.demand
                if customer.id == customer_in_cluster.id and new_remaining_capacity > 0:
                    assigned_customers.append(customer_in_cluster)
                    customers.remove(customer)
                    remaining_capacity = new_remaining_capacity
                    #print('[vehicle {0} - 1st assignment] remaining customers: {1}, remaining capacity: {2}'
                          #.format(vehicle.id, len(customers), remaining_capacity))
                    break
        #assign unassigned customers to the nearest vehicle
        # Calculate the Euclidean distance between unasigned customers and centroid of cluster(= centroid of vehicle)
        for customer in customers:
            dist = distance(customer, vehicle)
            ordered_customers_tuple.append(OrderedCustomer(dist, customer))

        # Sort by distance ascending(i.e. the nearest customers from vehicle)
        ordered_customers_tuple = sorted(ordered_customers_tuple, key=lambda x: x.distance)
        # Assign customers in the remaining by nearest distance order
        for ordered_customer in ordered_customers_tuple:
          customer = ordered_customer.data
          new_remaining_capacity = remaining_capacity - customer.demand
          if new_remaining_capacity < 0:
            break
          
          assigned_customers.append(customer)
          customers.remove(customer)
          remaining_capacity = new_remaining_capacity
          #print('[vehicle {0} - 2nd assignment] remaining customers: {1}, remaining capacity: {2}'
                     # .format(vehicle.id, len(customers), remaining_capacity))

        # Return new vehicle instance with new list of assigned customers
        vehicle_ = Vehicle(vehicle.id, vehicle.capacity, 0.0, vehicle.x, vehicle.y, assigned_customers, vehicle.initialDistance)
        #print('* vehicle[{0}]: assigned {1} customers'.format(vehicle.id, len(assigned_customers)))
        vehicles_.append(vehicle_)

        if len(customers) == 0:
            break

    # Assign the remaining customers to the nearest centroid(i.e. vehicle) if the vehicle capacity is available (one more time to make sure no customer is missing )
    #print('number of unassigned customers = %d' % len(customers))
    if len(customers) > 0:
        unassigned_customers = []
        for customer in customers:
            nearest_vehicle = None
            min_distance = np.inf
            for vehicle in vehicles_:
                dist = distance(customer, vehicle)
                if dist <= min_distance and customer.demand <= vehicle.capacity:
                    min_distance = dist
                    nearest_vehicle = vehicle
                    # print('min distance: %s' % str(min_distance))

            #print('nearest vehicle: %s' % str(nearest_vehicle.id + 1))
            if nearest_vehicle is not None:
                nearest_vehicle.customers.append(customer)
                unassigned_customers.append(customer)

        for customer in unassigned_customers:
            customers.remove(customer)

    #print('number of remaining customers = %d' % len(customers))

    return vehicles_

In [ ]:
def make_vehicle_route(depot, vehicle):
    
    #Optimizes the vehicle routing.
    
    points = []
    points.append(depot)
    for customer in vehicle.customers:
        points.append(customer)

    # Greedy solution (nearest neighbor)
    # Starts from 0, add nearest neighbor to the cycle at each step


    # 2-opt solution
    best_distance, opt, best_tour = two_opt(points)
    #print('* best distance: {0}'.format(str(best_distance)))
    #print('* best tour: {0}'.format(best_tour))

    # Calculate the cost of the solution
    cost = best_distance
    # cost += distance(depot, points[best_tour[0]])
    # cost += distance(depot, points[best_tour[len(best_tour) - 1]])

    #print('* total cost: {0}'.format(str(cost)))

    # Make directed cycle graph starting from the depot and returning to the depot.
    graph = []
    depot_index = len(best_tour)
    index = 0
    lefthand_vertices_of_depot = []
    for vertex in best_tour:
        if vertex == 0:  # Start from the depot
            depot_index = index
            graph.append(vertex)
        elif index > depot_index:
            graph.append(vertex)
        else:
            lefthand_vertices_of_depot.append(vertex)

        index += 1
    for vertex in lefthand_vertices_of_depot:
        graph.append(vertex)

    graph.append(0)  # Edge from the last vertex to the depot
    solution = []
    for vertex in graph:
        solution.append(points[vertex])
    return cost, opt, solution

In [ ]:
def two_opt(points):
    point_count = len(points)
    best_distance, _, best_tour = greedy(points)
    improved = True
    t = time.clock()
    while improved:
        improved = False
        for start, end in itertools.combinations(range(point_count), 2):
            curr_tour, curr_distance = swap(best_tour, best_distance, start, end, points)
            if curr_distance < best_distance:
                best_tour = curr_tour
                best_distance = curr_distance
                improved = True
                break
        if time.clock() - t >= 4 * 3600 + 59 * 60:
            improved = False
    return tour_distance(best_tour, points), 0, best_tour

def greedy(points):
    point_count = len(points)
    coords = np.array([(point.x, point.y) for point in points])
    tour = [0]
    candidates = set(range(1, point_count))
    while candidates:
        curr_point = tour[-1]
        nearest_neighbor = None
        nearest_dist = np.inf
        for neighbor in candidates:
            if LA.norm(coords[curr_point] - coords[neighbor]) < nearest_dist:
                nearest_neighbor = neighbor
                nearest_dist = LA.norm(coords[curr_point] - coords[neighbor])

        tour.append(nearest_neighbor)
        candidates.remove(nearest_neighbor)
    return tour_distance(tour, points), 0, tour

def swap(tour, dist, start, end, points):
    new_tour = tour[:start] + tour[start:end + 1][::-1] + tour[end + 1:]

    new_distance = dist - \
                   (distance(points[tour[start - 1]], points[tour[start]]) +
                    distance(points[tour[end]], points[tour[(end + 1) % len(tour)]])) + \
                   (distance(points[new_tour[start - 1]], points[new_tour[start]]) +
                    distance(points[new_tour[end]], points[new_tour[(end + 1) % len(tour)]]))
    return new_tour, new_distance

def tour_distance(tour, points):
    return sum(distance(points[tour[i - 1]], points[tour[i]]) for i in range(len(tour)))

In [ ]:
class Point:
  def __init__(self, x, y):
    self.x = float(x)
    self.y = float(y)

class Customer:
  def __init__(self, id, x, y, demand):
    self.id = id
    self.x = float(x)
    self.y = float(y)
    self.demand = int(demand)

class Vehicle:
  def __init__(self, id, capacity, cost, x, y, customers, initialDistance):
    self.id = id
    self.capacity = capacity
    self.cost = cost
    self.x = float(x)
    self.y = float(y)
    self.customers = customers
    self.initialDistance = initialDistance

class OrderedCustomer:
  def __init__(self, dist, data):
    self.distance = dist
    self.data = data

def main():
  depot = Point(data['depot'][0], data['depot'][1])
  capacity = data['capacity']

  # Create customers list
  customers = []
  for i in range(len(data['x'])):
    customer = Customer(data['id'][i], data['x'][i], data['y'][i], data['d'][i])
    customers.append(customer)
  
  # Calculate the minimum number of truck we need
  demands = [int(i) for i in data['d']]
  num_truck = sum(demands)//data['capacity'] + 1

  clusters, centroids = cluster_customers(num_truck, customers)

  vehicles = init_vehicles(depot, centroids, clusters, customers, capacity)

  vehicles = assign_customers_to_vehicles(customers, vehicles, capacity)
  #print(vehicles)

  # Make route plan and calculate cost
  output_data = ''
  total_cost = 0
  final_route = {}
  for vehicle in vehicles:
      cost, opt, vehicle_tour = make_vehicle_route(depot, vehicle)
      total_cost += cost
      #output_data += 'vehicle' + str(vehicle.id + 1) + ': ' + ' '.join([str(int(vertex)) for vertex in vehicle_tour]) + '\n'
      #print('* vehicle_tour: {0}'.format(vehicle_tour.))
      routes = []
      for i in range(len(vehicle_tour)):
        if hasattr(vehicle_tour[i], 'id'):
          routes.append(vehicle_tour[i].id)
          #routes['Vehicle'].append(vehicle.id)
      
      #print('Vehicle {0} has route {1}'.format(vehicle.id, routes))
      final_route[vehicle.id] = routes
      
  output_data = 'total cost: %.5f' % total_cost + '\n'

  return output_data, final_route

In [ ]:
output, final_route = main()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  from ipykernel import kernelapp as app


In [ ]:
with open('json_data.json', 'w') as outfile:
    json.dump(final_route, outfile)